In [216]:
import openpyxl
import pandas as pd
import numpy as np

# .. OBS Data load
read_dir = "D:\KMK_DATA\OBS\HOURLY\\"
read_list = [
    "47105_OBS_ASOS_TIM_20210101_20211231.csv",
    "47105_OBS_ASOS_TIM_20220101_20221231.csv",
    "47105_OBS_ASOS_TIM_20230101_20230630.csv"
]
set_col = ['지점', '지점명', '일시', '풍속(m/s)', '풍향(16방위)']

df1 = pd.DataFrame()
for file in read_list:
    read_file = read_dir + file
    #print(read_file)
    df = pd.read_csv(read_file, encoding='cp949')
    df1 = pd.concat([df1, df], axis=0, ignore_index=True)


df1 = df1[set_col]
df1['일시'] = pd.to_datetime(df1['일시'])
print(df1)




        지점 지점명                  일시  풍속(m/s)  풍향(16방위)
0      105  강릉 2021-01-01 00:00:00      4.7     230.0
1      105  강릉 2021-01-01 01:00:00      3.4     250.0
2      105  강릉 2021-01-01 02:00:00      2.8     230.0
3      105  강릉 2021-01-01 03:00:00      1.5     230.0
4      105  강릉 2021-01-01 04:00:00      3.4     230.0
...    ...  ..                 ...      ...       ...
21854  105  강릉 2023-06-30 19:00:00      0.3       0.0
21855  105  강릉 2023-06-30 20:00:00      1.1     140.0
21856  105  강릉 2023-06-30 21:00:00      0.5      70.0
21857  105  강릉 2023-06-30 22:00:00      0.9      50.0
21858  105  강릉 2023-06-30 23:00:00      0.5     320.0

[21859 rows x 5 columns]


In [217]:
from metpy.calc import wind_components
from metpy.units import units

#df['새로운변수'] = df.apply(lambda x: sample_function(x.변수1, x.변수2, ...), axis = 1)

def wind_to_uv(spd, dir):
    a = wind_components(spd * units('m/s'), dir * units.deg)
    return np.array(a[0]),np.array(a[1])

u_list, v_list = [], []
for i in range(df1.shape[0]):
    u, v = wind_to_uv(df1['풍속(m/s)'][i], df1['풍향(16방위)'][i])
    u_list.append(u)
    v_list.append(v)
#u, v = wind_to_uv(df1['풍속(m/s)'], df1['풍향(16방위)'])
#u.shape
df1['U_comp.'] = np.round(u_list, 2)
df1['V_comp.'] = np.round(v_list, 2)

df1 = df1.set_index('일시')

In [218]:
df1.info()
df1.tail()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21859 entries, 2021-01-01 00:00:00 to 2023-06-30 23:00:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   지점        21859 non-null  int64  
 1   지점명       21859 non-null  object 
 2   풍속(m/s)   21849 non-null  float64
 3   풍향(16방위)  21849 non-null  float64
 4   U_comp.   21849 non-null  float64
 5   V_comp.   21849 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.2+ MB


,지점,지점명,풍속(m/s),풍향(16방위),U_comp.,V_comp.
일시,,,,,,
2023-06-30 19:00:00,105,강릉,0.3,0.0,-0.00,-0.30
2023-06-30 20:00:00,105,강릉,1.1,140.0,-0.71,0.84
2023-06-30 21:00:00,105,강릉,0.5,70.0,-0.47,-0.17
2023-06-30 22:00:00,105,강릉,0.9,50.0,-0.69,-0.58
2023-06-30 23:00:00,105,강릉,0.5,320.0,0.32,-0.38


In [219]:
print(df1.info())
print(df1.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 21859 entries, 2021-01-01 00:00:00 to 2023-06-30 23:00:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   지점        21859 non-null  int64  
 1   지점명       21859 non-null  object 
 2   풍속(m/s)   21849 non-null  float64
 3   풍향(16방위)  21849 non-null  float64
 4   U_comp.   21849 non-null  float64
 5   V_comp.   21849 non-null  float64
dtypes: float64(4), int64(1), object(1)
memory usage: 1.2+ MB
None
지점           0
지점명          0
풍속(m/s)     10
풍향(16방위)    10
U_comp.     10
V_comp.     10
dtype: int64


In [236]:
# 결측치의 index 번호 찾기
mask = df1['U_comp.'].isnull()
index_list = df1.index[mask]

print(index_list)

#print(df1.index)
# with open('./print_dates.txt', 'w', encoding='cp949') as f:
#     for i in df1.index:
#         print(i, file=f)

DatetimeIndex(['2022-03-19 09:00:00', '2022-03-19 10:00:00',
               '2022-03-19 11:00:00', '2022-07-13 14:00:00',
               '2022-09-08 06:00:00', '2022-09-08 07:00:00',
               '2022-09-08 08:00:00', '2022-09-08 09:00:00',
               '2022-09-08 10:00:00', '2022-09-08 11:00:00'],
              dtype='datetime64[ns]', name='일시', freq=None)


In [ ]:
#df1.loc['2022-03-19 09:00:00']
df1.loc['2021-10-27 01:00:00'] # 결측 NaN, 진짜 없음 결측
#df1.loc['2021-12-31 02:00:00']

In [235]:
# .. extract date
set_dates = pd.date_range('2021-01-14 09:00:00', '2023-05-31 09:00:00', freq='D')
print("있어야할 day수: ", len(set_dates))


# rdates = pd.date_range('2022-03-19 09:00:00', periods=48, freq='H')
# print('1111', set_dates[0:2])


# make (batch, sequence=48, feature=[u,v])
obs_data = np.full( (len(set_dates),48,2), np.nan )


for i,day in enumerate(set_dates):
   rdates = pd.date_range(day, periods=48, freq='H')
   try:
      for j, rdate in enumerate(rdates):
         obs_data[i,:,0] = df1.loc[rdate,'U_comp.']
         obs_data[i,:,1] = df1.loc[rdate,'V_comp.']
   except Exception as e:
      print(e)  # nan으로 있지도 않은 자료


있어야할 day수:  868
Timestamp('2021-10-27 01:00:00')
Timestamp('2021-10-27 01:00:00')
Timestamp('2022-08-26 01:00:00')
Timestamp('2022-08-26 01:00:00')


In [237]:
rdates = pd.date_range('2022-05-31 09:00:00', periods=48, freq='H')
print(rdates)
test_spd = df1.loc[rdates,'풍속(m/s)']
test_dir = df1.loc[rdates,'풍향(16방위)']

DatetimeIndex(['2022-05-31 09:00:00', '2022-05-31 10:00:00',
               '2022-05-31 11:00:00', '2022-05-31 12:00:00',
               '2022-05-31 13:00:00', '2022-05-31 14:00:00',
               '2022-05-31 15:00:00', '2022-05-31 16:00:00',
               '2022-05-31 17:00:00', '2022-05-31 18:00:00',
               '2022-05-31 19:00:00', '2022-05-31 20:00:00',
               '2022-05-31 21:00:00', '2022-05-31 22:00:00',
               '2022-05-31 23:00:00', '2022-06-01 00:00:00',
               '2022-06-01 01:00:00', '2022-06-01 02:00:00',
               '2022-06-01 03:00:00', '2022-06-01 04:00:00',
               '2022-06-01 05:00:00', '2022-06-01 06:00:00',
               '2022-06-01 07:00:00', '2022-06-01 08:00:00',
               '2022-06-01 09:00:00', '2022-06-01 10:00:00',
               '2022-06-01 11:00:00', '2022-06-01 12:00:00',
               '2022-06-01 13:00:00', '2022-06-01 14:00:00',
               '2022-06-01 15:00:00', '2022-06-01 16:00:00',
               '2022-06-

In [238]:
# np.array 결측 제거 테스트
# a = np.array([[1,2,3],[4,5,6],[7,8,np.nan]])
# print(a.shape)
# mask = np.isnan(a).any(axis=0)
# b = a[~mask]
# print(b.shape)



from metpy.calc import wind_speed, wind_direction
print(obs_data.shape)


# 원래 파일에도 없는 index때문에 배열 꼬인듯, 고치기
for i, (u, v) in enumerate(obs_data[-1,:,:]):
    spd = np.array(np.round(wind_speed(u* units('m/s'), v* units('m/s')), 2))
    dir = np.array(np.round(wind_direction(u* units('m/s'),v* units('m/s')), 0))
    print(i, spd, test_spd[i], dir, test_dir[i], sep=',  ' )
    





(868, 48, 2)
0,  1.5,  3.6,  90.0,  320.0
1,  1.5,  5.0,  90.0,  320.0
2,  1.5,  2.4,  90.0,  70.0
3,  1.5,  1.8,  90.0,  90.0
4,  1.5,  2.4,  90.0,  70.0
5,  1.5,  1.5,  90.0,  90.0
6,  1.5,  1.8,  90.0,  90.0
7,  1.5,  2.2,  90.0,  90.0
8,  1.5,  5.1,  90.0,  250.0
9,  1.5,  4.0,  90.0,  230.0
10,  1.5,  2.9,  90.0,  250.0
11,  1.5,  3.5,  90.0,  250.0
12,  1.5,  2.2,  90.0,  250.0
13,  1.5,  3.1,  90.0,  250.0
14,  1.5,  3.5,  90.0,  200.0
15,  1.5,  2.8,  90.0,  230.0
16,  1.5,  2.8,  90.0,  200.0
17,  1.5,  3.0,  90.0,  250.0
18,  1.5,  5.1,  90.0,  250.0
19,  1.5,  2.9,  90.0,  250.0
20,  1.5,  2.5,  90.0,  290.0
21,  1.5,  2.4,  90.0,  230.0
22,  1.5,  2.9,  90.0,  250.0
23,  1.5,  1.6,  90.0,  230.0
24,  1.5,  1.8,  90.0,  230.0
25,  1.5,  4.5,  90.0,  230.0
26,  1.5,  3.9,  90.0,  230.0
27,  1.5,  4.6,  90.0,  250.0
28,  1.5,  2.2,  90.0,  70.0
29,  1.5,  2.3,  90.0,  90.0
30,  1.5,  1.9,  90.0,  70.0
31,  1.5,  1.5,  90.0,  70.0
32,  1.5,  2.3,  90.0,  50.0
33,  1.5,  1.2,  9